# Experiment 4
- Load the saved model and replace the output layer of the model, as well as the two last convolutional layers.
- Train and evaluate the model on the cats and dogs dataset.

## Steps
- Load the saved model using keras.models.load_model().
- Freeze all layers except the output layer and the last two convolutional layers.
- Replace the output layer to match the number of classes (num_classes = 2 for cats vs. dogs).
- Retrain the model on the Cats vs. Dogs dataset.

### Load the saved model

In [1]:
import tensorflow as tf
from tensorflow import data as tf_data
from tensorflow import keras
from keras import layers

# Load the pre-trained model
saved_model_path = "models/experiment1_model.keras"
model = keras.models.load_model(saved_model_path)

### Modify the model

In [ ]:
# Unfreeze the last two convolutional layers
for layer in model.layers[-4:]:  # Assuming the last 4 layers include the 2 Conv layers and BatchNorm
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.SeparableConv2D):
        layer.trainable = True

# Modify the output layer
num_classes = 2  # Binary classification (Cats vs. Dogs)
x = model.layers[-2].output  # Take output from the second-last layer
x = layers.Dense(num_classes, activation="softmax")(x)  # New classification layer

# Create new model
new_model = keras.Model(inputs=model.input, outputs=x)

### Compile the model

In [ ]:
new_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


### Train and evaluate the model

In [ ]:
# Create the dataset
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x), y))

# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

In [ ]:
epochs = 50
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="models/experiment3_epoch_{epoch}.keras", period=10)
]

new_model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=callbacks
)

new_model.save("experiment3_model.keras")
